#### accepts.csv 数据集描述与属性说明
- 数据说明：本数据是一份汽车贷款违约数据,因变量为是否违约(bad_ind)
- [T]因变量,[N]数值变量,[C]分类变量,[S]序数变量,[D]时间
- application_id	申请者ID
- account_number	帐户号
- [T]bad_ind	是否违约
- [D]vehicle_year	汽车购买时间
- [C]vehicle_make	汽车制造商
- [C]bankruptcy_ind	曾经破产标识
- [N]tot_derog	五年内信用不良事件数量(比如手机欠费消号)
- [N]tot_tr	全部帐户数量
- [N]age_oldest_tr	最久账号存续时间(月)
- [N]tot_open_tr	在使用帐户数量
- [N]tot_rev_tr	在使用可循环贷款帐户数量(比如信用卡)
- [N]tot_rev_debt	在使用可循环贷款帐户余额(比如信用卡欠款)
- [N]tot_rev_line	可循环贷款帐户限额(信用卡授权额度)
- [N]rev_util	可循环贷款帐户使用比例(余额/限额)
- [N]fico_score	FICO打分
- [N]purch_price	汽车购买金额(元)
- [N]msrp	建议售价
- [N]down_pyt	分期付款的首次交款
- [N]loan_term	贷款期限(月)
- [N]loan_amt	贷款金额
- [N]ltv	贷款金额/建议售价*100
- [N]tot_income	月均收入(元)
- [N]veh_mileage	行使历程(Mile)
- [C]used_ind	是否使用
- [N]weight	样本权重

# 数据整合和数据清洗
- pandas学习参考： [十分钟搞定pandas](http://www.cnblogs.com/chaosimple/p/4153083.html)

## SQL语句介绍

In [11]:
import os

# os.chdir('E:/data')

In [12]:
import pandas as pd
import numpy as np

sale = pd.read_csv('sale.csv', encoding='gbk')
sale.head()

,year,market,sale,profit
0,2010,东,33912,2641
1,2010,南,32246,2699
2,2010,西,34792,2574
3,2010,北,31884,2673
4,2011,东,31651,2437


- SQL2数据过滤与排序
- 选择表中指定列

In [13]:
import sqlite3 # sqlite3相当于轻量版，更多功能可使用SQLAlchemy

con = sqlite3.connect(':memory:') # 数据库连接  con = sqlite3.connect('mydata.db')
sale.to_sql('sale', con) # 将DataFrame注册成可用sql查询的表
newTable = pd.read_sql_query("select year , sale, market,profit from sale", con) # 也可使用read_sql
newTable.head()

,year,sale,market,profit
0,2010,33912,东,2641
1,2010,32246,南,2699
2,2010,34792,西,2574
3,2010,31884,北,2673
4,2011,31651,东,2437


- 选择表中所有列

In [14]:
sqlResult = pd.read_sql_query('select * from sale', con)
sqlResult

,index,year,market,sale,profit
0,0,2010,东,33912,2641
1,1,2010,南,32246,2699
2,2,2010,西,34792,2574
3,3,2010,北,31884,2673
4,4,2011,东,31651,2437
5,5,2011,南,30572,2853
6,6,2011,西,34175,2877
7,7,2011,北,30555,2749
8,8,2012,东,31619,2106
9,9,2012,南,32443,3124


- 删除重复的行

In [15]:
pd.read_sql_query("select DISTINCT  year, market from sale", con)

,year,market
0,2010,东
1,2010,南
2,2010,西
3,2010,北
4,2011,东
5,2011,南
6,2011,西
7,2011,北
8,2012,东
9,2012,南


- 选择满足条件的行

In [19]:
pd.read_sql_query("select * from sale where year=2012 and market='东'", con)

,index,year,market,sale,profit
0,8,2012,东,31619,2106


- 对行进行排序

In [20]:
sql = '''select year, market, sale, profit
      from sale
      order by year'''
pd.read_sql_query(sql, con)
sale[(sale['year']>2011) & (sale['profit']<3000)]
(sale['year']>2011) & (sale['profit']<3000)
sale[(sale['market'].isin(['东','南'])) & (sale['profit']<9000)]

sale.sort_values('profit')
sale['profit'].rank(axis=0)

0      5.0
1      7.0
2      3.0
3      6.0
4      2.0
5      9.0
6     10.0
7      8.0
8      1.0
9     12.0
10     4.0
11    11.0
Name: profit, dtype: float64

##  纵向连接表
sql操作

In [22]:
one = pd.read_csv("one.csv")
one.to_sql('one', con, index=False)
print(one)
print(one.T)
one.T

   x  a
0  1  a
1  1  a
2  1  b
3  2  c
4  3  v
5  4  e
6  6  g
   0  1  2  3  4  5  6
x  1  1  1  2  3  4  6
a  a  a  b  c  v  e  g


,0,1,2,3,4,5,6
x,1,1,1,2,3,4,6
a,a,a,b,c,v,e,g


In [23]:
two = pd.read_csv("two.csv")
two.to_sql('two', con, index=False)
two.T

,0,1,2,3,4
x,1,2,3,3,5
b,x,y,z,v,w


union 和 union all

In [24]:
#去重复
union = pd.read_sql('select * from one UNION select * from two', con)
print(union)
#不去重复
union_all = pd.read_sql('select * from one UNION ALL select * from two', con)
print(union_all)
union.T

   x  a
0  1  a
1  1  b
2  1  x
3  2  c
4  2  y
5  3  v
6  3  z
7  4  e
8  5  w
9  6  g
    x  a
0   1  a
1   1  a
2   1  b
3   2  c
4   3  v
5   4  e
6   6  g
7   1  x
8   2  y
9   3  z
10  3  v
11  5  w


,0,1,2,3,4,5,6,7,8,9
x,1,1,1,2,2,3,3,4,5,6
a,a,b,x,c,y,v,z,e,w,g


In [25]:
union_all.T

,0,1,2,3,4,5,6,7,8,9,10,11
x,1,1,1,2,3,4,6,1,2,3,3,5
a,a,a,b,c,v,e,g,x,y,z,v,w


except 和 intersect

In [26]:
#差集
exceptTable = pd.read_sql('select * from one EXCEPT select * from two', con)
#交集
intersectTable = pd.read_sql('select * from one INTERSECT select * from two', con)
exceptTable.T

,0,1,2,3,4
x,1,1,2,4,6
a,a,b,c,e,g


In [27]:
intersectTable.T

,0
x,3
a,v


*练习： 多表纵向连接

DataFrame操作

In [28]:
pd.concat([one, two], axis=0, join='outer', ignore_index=True) # 更多参数可查看文档或帮助

,a,b,x
0,a,NaN,1
1,a,NaN,1
2,b,NaN,1
3,c,NaN,2
4,v,NaN,3
5,e,NaN,4
6,g,NaN,6
7,NaN,x,1
8,NaN,y,2
9,NaN,z,3


##   横向连接表
sql操作

In [ ]:
table1 = pd.read_csv('table1.csv')
table1.to_sql('table1', con, index=False)
table1.head()

In [ ]:
table2 = pd.read_csv('table2.csv')
table2.to_sql('table2', con, index=False)
table2.head()

笛卡尔积

In [ ]:
pd.read_sql("select * from table1, table2", con)
table1.join(table2,lsuffix='left')

内连接（使用inner join或使用where子句）

In [ ]:
#inner join
pd.read_sql("select * from table1 as a inner join table2 as b on a.id=b.id", con)
# pd.read_sql("select * from table1 as a, table2 as b where a.id=b.id", con)

左连接

In [ ]:
#left join
pd.read_sql("select * from table1 as a left join table2 as b on a.id=b.id", con)

In [ ]:
# Attention: RIGHT and FULL OUTER JOINs are NOT currently supported, like:
# "select * from table1 as a RIGHT JOIN table2 as b on a.id=b.id"
# "select * from table1 as a FULL JOIN table2 as b on a.id=b.id"
#left join
pd.read_sql("select * from table1 as a FULL JOIN table2 as b on a.id=b.id", con)

DataFrame操作

In [ ]:
pd.merge(table1, table2, on='id', how='left') # 参数设置可查看帮助

按索引连接

In [ ]:
#按照？拼接
table1.join(table2, how='inner', lsuffix='t1', rsuffix='t2') # 参数设置可查看帮助

In [ ]:
#按照索引拼接
table1.join(table2, how='outer', lsuffix='t1', rsuffix='t2') # 参数设置可查看帮助

排序

In [ ]:
pd.read_sql("select * from table2 order by id", con)

### apply\map\groupby及其它相关

In [ ]:
data = pd.DataFrame(data={'a':range(1,11), 'b':np.random.randn(10)})
data.T

In [ ]:
data.apply(np.mean) # 等价于data.mean()，是其完整形式

In [ ]:
data.apply(lambda x: x.astype('str')).dtypes # DataFrame没有astype方法，只有Series有

In [ ]:
(data['a']- data['a'].mean()) / data['a'].std()

In [ ]:
#标准化
data['a'].map(lambda x: (x - data['a'].mean()) / data['a'].std())  # 等价于(data['a']- data['a'].mean()) / data['a'].std()

In [ ]:
data['a'].map(lambda x: int(str(x), base=16))  # 不支持“广播”时，可以使用map进行函数映射

分组-应用/聚合

In [ ]:
key = [1, 2] * 5
key

In [ ]:
group1, group2 = data.groupby(key) # 使用groupby可按照‘key’进行分组，‘key’需与待分组数据有同样长度
print(group1)
print(group2)

In [ ]:
data.groupby(key).aggregate(np.mean) 
data.groupby(key).mean()
# 聚合函数在各分组中进行聚合，是data.groupby(key).mean()的完整形式，可传入函数或字符串(sum/mean/median/std/var等)，也可传入列表

In [ ]:
data.groupby(key).agg({'a': 'std', 'b':'std'}) # agg可以在多列上使用不同的聚合函数

In [ ]:
data.groupby(key).agg({'a': 'sum', 'b':'count'}) # agg可以在多列上使用不同的聚合函数

In [ ]:
data.groupby(key).transform(np.mean) # 转换函数可在各分组内进行运算，将结果广播到原数据中

In [ ]:
data.groupby(key).apply(np.mean) # apply是一般化的‘分组-应用/聚合’函数，更灵活地实现aggregate或transform的功能

*练习：对accepts数据集，按照是否破产汇总违约率



In [ ]:
import pandas as pd

accepts = pd.read_csv('accepts.csv')

按照违约指示变量进行分组汇总

In [ ]:
print(accepts['bad_ind'].groupby(accepts['bankruptcy_ind']).mean())


vmean = accepts.fico_score.mean(skipna=True)
accepts['fico_score'] = accepts['fico_score'].fillna(vmean)
accepts['fico_score'].groupby(accepts['bad_ind']).apply(lambda x: x.describe())
# accepts['fico_score'].groupby(accepts['bad_ind']).describe()

In [ ]:
print(accepts[['purch_price','fico_score']])
print(accepts[['purch_price','fico_score']].groupby([accepts['bad_ind']]).mean())
# 按照['bad_ind', 'bankruptcy_ind']两个关键字段汇总['purch_price','fico_score']的均值
accepts[['purch_price','fico_score']].groupby([accepts['bad_ind'], accepts['bankruptcy_ind']]).mean()

crosstab 和pivot_table

In [ ]:
pd.crosstab(accepts.bad_ind, accepts.bankruptcy_ind)

In [ ]:
accepts.pivot_table(['purch_price', 'fico_score'], 
                    index='bad_ind', columns='bankruptcy_ind', aggfunc=np.mean) # index、columns、aggfunc参数均可传入列表

## 数据清洗

发现数据问题类型

In [ ]:
accepts = pd.read_csv('accepts.csv')
accepts.head()

In [ ]:
pd.set_option('display.max_columns', None) # 设置显示所有列
# Try this: pd.describe_option('display')

- 脏数据或数据不正确

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
#直方图
plt.hist(accepts['purch_price'], bins=20, normed=True)
# Try this: accepts['purch_price'].plot(kind='hist')
# And this: sns.distplot(accepts['purch_price'], kde=True, fit=stats.norm) #should import seaborn as sns and scipy.stats first

In [ ]:
accepts['purch_price'].describe(include='all')

In [ ]:
def f(x, n=100):
    if x < n:
        x = np.nan
    return x

accepts['purch_price'] = accepts['purch_price'].map(f)
# Or try: accepts['purch_price'][accepts['purch_price'] < 100] = np.nan # It causes a Warning
accepts.purch_price.describe() # It will not count 'nan' value

- 数据不一致-
这个问题需要详细的结合描述统计进行变量说明核对

- 数据重复

In [ ]:
accepts['dup'] = accepts.duplicated() # 生成重复标识变量
accepts.dup.head()

In [ ]:
accepts_dup = accepts[accepts['dup'] == True] # 把有重复的数据保存出来，以备核查
accepts_nodup = accepts[accepts['dup'] == False] # 注意与accepts.drop_duplicates()的区别
accepts_nodup.head()

In [ ]:
accepts['dup1'] = accepts['account_number'].duplicated() # 按照主键进行重复记录标识
# accepts['fico_score'].duplicated() # 没有实际意义

* 缺失值处理

In [ ]:
accepts.describe()

In [ ]:
vmean = accepts['tot_derog'].mean(axis=0, skipna=True)
accepts['tot_derog_empflag'] = accepts['tot_derog'].isnull()
accepts['tot_derog'] = accepts['tot_derog'].fillna(vmean)
accepts.tot_derog.describe()

- 其他有缺失变量请自行填补，找到一个有缺失的分类变量，使用众数进行填补
- 多重插补：sklearn.preprocessing.Imputer仅可用于填补均值、中位数、众数，多重插补可考虑使用Orange、impute、Theano等包
- 多重插补的处理有两个要点：1、被解释变量有缺失值的观测不能填补，只能删除；2、只对放入模型的解释变量进行插补。

* 噪声值处理
- 盖帽法

In [ ]:
def blk(floor, root): # 'blk' will return a function
    def f(x):       
        if x < floor:
            x = floor
        elif x > root:
            x = root
        return x
    return f

q1 = accepts['tot_derog'].quantile(0.01) # 计算百分位数
q99 = accepts['tot_derog'].quantile(0.99)
blk_tot = blk(floor=q1, root=q99) # 'blk_tot' is a function
accepts['tot_derog'] = accepts['tot_derog'].map(blk_tot)
accepts['tot_derog'].describe()

- k-means聚类法、分箱（等深，等宽）
- 分箱法——等宽分箱

In [ ]:
accepts['age_oldest_tr_1'] = pd.qcut(accepts['age_oldest_tr'], 4) # 这里以age_oldest_tr字段等宽分为4段
accepts.age_oldest_tr_1.head()

- 分箱法——等深分箱

In [ ]:
accepts['age_oldest_tr_2'] = pd.cut(accepts['age_oldest_tr'], 4) # 这里以age_oldest_tr字段等比分为4段
# Try this: pd.cut(accepts['age_oldest_tr'], [0, 148, 295, 442, 588 ]) # 自定义分箱  [-np.nan, 148, 295, 442, np.nan ]
accepts.age_oldest_tr_2.head()